In [314]:
import os
import numpy as np
import pandas as pd
import geopandas as gp
from datetime import datetime
import json

import shapely
from fiona.crs import from_epsg
import psycopg2

import matplotlib.pyplot as plt
%matplotlib inline

In [304]:
# Source of data - https://www.baruch.cuny.edu/confluence/pages/viewpage.action?pageId=28016896
def download_data():
    os.system("curl -O http://faculty.baruch.cuny.edu/geoportal/data/nyc_transit/may2019/bus_routes_nyc_may2019.zip")
    os.system("mkdir Data")
    os.system("unzip bus_routes_nyc_may2019.zip -d ./Data/bus_routes_nyc_may2019/")

In [305]:
## https://gis.stackexchange.com/questions/309251/how-to-get-equidistant-points-from-a-linestring-geographical-coordinates

def create_geo(busline, tpoints ,time_index):
    '''
    Doc: busline: row indicating busline, should contain geometry
          time_index = pd.date_range obj with required cadance
    '''
    busLineFinal = []
    totalIdx = len(time_index)
    lastIdx = 0
    flag = True
    
    
    route_dir = busline['route_dir']
    route_id = busline['route_id']
    
    #Clip line into 250 unique points
    num_points = tpoints
    points = [busline['geometry'].interpolate(i/float(num_points - 1),
                                                                 normalized=True)
                                                     for i in range(num_points)] 

    busline_geom = gp.GeoSeries(points)
    while flag:
        
        busLine = gp.GeoDataFrame(geometry=busline_geom)
        busLine['route_name'] = route_id
        busLine['route_dir'] = route_dir
        busLine['time'] = np.NaN  
        
        for idx in busLine.index:
            busLine.loc[idx, 'time'] = time_index[lastIdx]
            lastIdx = lastIdx + 1
   
            if lastIdx == totalIdx:
                flag = False
                break
                
        busLineFinal.append(busLine)
    
    return (pd.concat(busLineFinal).dropna().reset_index(drop=True)).to_dict()

## Download Bus route shape file

In [ ]:
download_data()

## Create a dataframe by splitting Brooklyn Bus routes in 250 points and for starting 121 points add timestamps of every thirty secs between 10AM to 11AM on 14th Aug 2019 

In [306]:
gdf = gp.read_file('./Data/bus_routes_nyc_may2019/bus_routes_nyc_may2019.shp')

In [308]:
gdf.head(2)

,color,dir_id,geometry,route_dir,route_id,route_long,route_shor
0,#B933AD,0,(LINESTRING (996119.7547271665 160954.75802325...,B100_0,B100,Mill Basin - Midwood,B100
1,#B933AD,1,(LINESTRING (1001037.784615552 160426.83236815...,B100_1,B100,Mill Basin - Midwood,B100


In [307]:
gdf.shape

(505, 7)

In [229]:
gdf.loc[(gdf.route_id.str.startswith("B")) & 
        (~gdf.route_id.str.contains("BX"))].shape

(115, 7)

In [230]:
points = 250
brooklyn_lines = gdf.loc[(gdf.route_id.str.startswith("B")) & 
                         (~gdf.route_id.str.contains("BX"))].copy()
bkPoints = brooklyn_lines.apply(lambda x: create_geo(x, 
                                                     points, 
                                                     pd.date_range('10:00', '11:00', freq='30s')), 
                                axis=1)

dataframer = lambda x: pd.DataFrame(x)
brooklynPoints_df = list(map(dataframer, bkPoints.tolist()));

brooklynPoints = pd.concat(brooklynPoints_df, ignore_index=True)

In [233]:
brooklynPoints.shape

(13915, 4)

In [240]:
brooklynPoints.head(3)

,geometry,route_dir,route_name,time
0,POINT (996119.7547271665 160954.7580232588),B100_0,B100,2019-08-14 10:00:00
1,POINT (996192.2380531991 160944.5904073866),B100_0,B100,2019-08-14 10:00:30
2,POINT (996281.488913448 160957.3293115214),B100_0,B100,2019-08-14 10:01:00


In [241]:
brooklynPoints = gp.GeoDataFrame(brooklynPoints, geometry='geometry')
brooklynPoints.crs = from_epsg(2263)
brooklynPoints.to_crs(epsg=4326, inplace=True)

In [244]:
brooklynPoints.head(2)

,geometry,route_dir,route_name,time
0,POINT (-73.957251 40.60845199999983),B100_0,B100,2019-08-14 10:00:00
1,POINT (-73.95698996884438 40.60842399459272),B100_0,B100,2019-08-14 10:00:30


In [309]:
brooklynBuses = brooklynPoints[['time','route_name', 'route_dir', 'geometry']].copy()
brooklynBuses.loc[:, 'lon'] = brooklynPoints.geometry.apply(lambda x: x.x)
brooklynBuses.loc[:, 'lat'] = brooklynPoints.geometry.apply(lambda x: x.y)
brooklynBuses.drop('geometry', axis=1, inplace=True)
brooklynBuses.head()

,time,route_name,route_dir,lon,lat
0,2019-08-14 10:00:00,B100,B100_0,-73.957251,40.608452
1,2019-08-14 10:00:30,B100,B100_0,-73.956990,40.608424
2,2019-08-14 10:01:00,B100,B100_0,-73.956669,40.608459
3,2019-08-14 10:01:30,B100,B100_0,-73.956347,40.608494
4,2019-08-14 10:02:00,B100,B100_0,-73.956026,40.608529


POPULATE bus.device_test table with DUMMY DATA


Insert into the following table using attached shapefile (points along major Brooklyn Bus routes) where device_id = route_id
you will need to create datetime such that each bus route starts at the same time and each point along the time represents an increment of 30 seconds.
Copy this dummy data to cover all "times of day" so that we can load the dummy data into the interface
insert into bus.device (datetime, device_id, lat, lon) VALUES (%s, %s, %s, %s);

In [327]:
keys = json.load(open("./credentials.json"))

host = keys['postgres'][0]['host']
database = keys['postgres'][0]['database']
user = keys['postgres'][0]['user']
password = keys['postgres'][0]['password']

con = psycopg2.connect(host=host, database=database, user=user, password=password)
cur = con.cursor()

In [302]:
query = 'insert into bus.device_test (datetime, device_id, lat, lon, route_id) VALUES (%s, %s, %s, %s, %s);'
for idx, row in brooklynBuses.iterrows():
    cur.execute(query,
                (datetime.strptime(str(row['time']), '%Y-%m-%d %H:%M:%S'), 
                 row['route_dir'],
                 row['lat'], row['lon'],
                 row['route_dir']))
con.commit()
# con.close()

In [328]:
query = "SELECT * FROM bus.device_test LIMIT 10;"

In [329]:
pd.read_sql(sql=query, con=con)

,lon,lat,device_id,datetime,geom,route_id
0,-73.957251,40.60845199999983,B100_0,2019-08-14 10:00:00,0101000020E61000000DC4B299437D52C0B09750C1E14D...,B100_0
1,-73.95698996884438,40.60842399459272,B100_0,2019-08-14 10:00:30,0101000020E610000059B2DA523F7D52C04F7063D6E04D...,B100_0
2,-73.94863443804769,40.6093416264626,B100_0,2019-08-14 10:13:30,0101000020E61000008CD1376DB67C52C0C9DF0AE8FE4D...,B100_0
3,-73.94831299573353,40.609376584471605,B100_0,2019-08-14 10:14:00,0101000020E610000007C3FD28B17C52C07AA04A0D004E...,B100_0
4,-73.94799141308184,40.609410790695655,B100_0,2019-08-14 10:14:30,0101000020E6100000D2042DE4AB7C52C005EF3B2C014E...,B100_0
5,-73.94766986621181,40.609445193122426,B100_0,2019-08-14 10:15:00,0101000020E610000034B2829FA67C52C02695D24C024E...,B100_0
6,-73.94734818765,40.609478852265255,B100_0,2019-08-14 10:15:30,0101000020E61000006CF84A5AA17C52C09C0A2D67034E...,B100_0
7,-73.94702616914623,40.6095105911042,B100_0,2019-08-14 10:16:00,0101000020E61000001C3CA6139C7C52C0D8AD6B71044E...,B100_0
8,-73.94670415033629,40.60954232904085,B100_0,2019-08-14 10:16:30,0101000020E6100000A32B01CD967C52C00A61A87B054E...,B100_0
9,-73.94638297417148,40.609578397504684,B100_0,2019-08-14 10:17:00,0101000020E6100000B2E3E489917C52C0BBD038AA064E...,B100_0
